In [ ]:
from langgraph.graph import StateGraph , START , END
from langchain import runnables
from langchain_huggingface import ChatHuggingFace , HuggingFaceEndpoint , HuggingFacePipeline
import os
from typing import TypedDict
from pydantic import BaseModel , from langgraph.graph import StateGraph , START , END
from langchain import runnables
from langchain_huggingface import ChatHuggingFace , HuggingFaceEndpoint , HuggingFacePipeline
import os
from typing import TypedDict
from pydantic import BaseModel , Field
from langchain.output_parsers import StructuredOutputParser , PydanticOutputParser
import json
from langchain.prompts import PromptTemplate

api_key = os.getenv('HUGGINGFACEHUB_API_KEY')

llm = HuggingFaceEndpoint(
    repo_id = "google/gemma-2-2b-it",
    task= "text-generation",
    huggingfacehub_api_token= api_key,  # ✅ THIS IS CRITICAL!

)

model = ChatHuggingFace(llm = llm)Field
from langchain.output_parsers import StructuredOutputParser , PydanticOutputParser
import json
from langchain.prompts import PromptTemplate

In [60]:
api_key = os.getenv('HUGGINGFACEHUB_API_KEY')

llm = HuggingFaceEndpoint(
    repo_id = "tiiuae/falcon-7b-instruct",
    task= "text-generation",
    huggingfacehub_api_token=api_key,  # ✅ THIS IS CRITICAL!

)

model = ChatHuggingFace(llm = llm)

In [61]:
from typing import Optional
from typing import TypedDict, Annotated
# --- Main essay state (input/output) ---
import operator
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedbackfor the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

parser = PydanticOutputParser(pydantic_object = EvaluationSchema)

prompt = PromptTemplate(
    template="""
You are an expert language evaluator. Your task is to evaluate the following essay and provide:

1. Detailed feedback on its structure, clarity, coherence, grammar, and overall quality.
2. A score out of 10 based on the quality.

Essay:
\"\"\"
{essay}
\"\"\"

{format_instructions}
""",
    input_variables=["essay"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

essay = """India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into their global services. In 2020, the government launched the National AI Strategy (AI for All) with a focus on inclusive growth, aiming to leverage AI in healthcare, agriculture, education, and smart mobility.

One of the most promising applications of AI in India lies in agriculture, where predictive analytics can guide farmers on optimal sowing times, weather forecasts, and pest control. In healthcare, AI-powered diagnostics can help address India’s doctor-patient ratio crisis, particularly in rural areas. Educational platforms are increasingly using AI to personalize learning paths, while smart governance tools are helping improve public service delivery and fraud detection.

However, the path to AI-led growth is riddled with challenges. Chief among them is the digital divide. While metropolitan cities may embrace AI-driven solutions, rural India continues to struggle with basic internet access and digital literacy. The risk of job displacement due to automation also looms large, especially for low-skilled workers. Without effective skilling and re-skilling programs, AI could exacerbate existing socio-economic inequalities.

Another pressing concern is data privacy and ethics. As AI systems rely heavily on vast datasets, ensuring that personal data is used transparently and responsibly becomes vital. India is still shaping its data protection laws, and in the absence of a strong regulatory framework, AI systems may risk misuse or bias.

To harness AI responsibly, India must adopt a multi-stakeholder approach involving the government, academia, industry, and civil society. Policies should promote open datasets, encourage responsible innovation, and ensure ethical AI practices. There is also a need for international collaboration, particularly with countries leading in AI research, to gain strategic advantage and ensure interoperability in global systems.

India’s demographic dividend, when paired with responsible AI adoption, can unlock massive economic growth, improve governance, and uplift marginalized communities. But this vision will only materialize if AI is seen not merely as a tool for automation, but as an enabler of human-centered development.

In conclusion, India in the age of AI is a story in the making — one of opportunity, responsibility, and transformation. The decisions we make today will not just determine India’s AI trajectory, but also its future as an inclusive, equitable, and innovation-driven society."""



structured_model = prompt | llm | parser



class UPSCState(TypedDict):

    essay: str
    language_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float


    clarity_feedback: Optional[str]
    clarity_score: Optional[int]
    analysis_feedback: Optional[str]
    analysis_score: Optional[int]
    thought_feedback: Optional[str]
    thought_score: Optional[int]
    final_result: Optional[str]


def evaluate_thought(state: UPSCState):
    print("🧠 Running evaluate_thought...")
    print("Input essay:", state.get("essay"))
    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
    output = structured_model.invoke(prompt)
    print("Output:", output)
    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}



In [62]:



def evaluate_analysis(state: UPSCState):
    try:
        prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {state["essay"]}'
        output = structured_model.invoke(prompt)

        return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}
    except Exception as e:
        print(f"Error in evaluate_language: {e}")
        raise
def evaluate_language(state: UPSCState):
    try:
        prompt = f"Evaluate the language quality: {state['essay']}"
        output = structured_model.invoke(prompt)
        return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

    except Exception as e:
        print(f"Error in evaluate_language: {e}")
        raise

In [63]:



def final_evaluation(state: UPSCState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}


In [64]:
graph = StateGraph(UPSCState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

initial_state = {
    "essay": essay  # ✅ Only the essay, nothing else
}

final_output = workflow.invoke(initial_state)
print(final_output)


Error in evaluate_language: 
Error in evaluate_language: 
🧠 Running evaluate_thought...
Input essay: India in the Age of AI
As the world enters a transformative era defined by artificial intelligence (AI), India stands at a critical juncture — one where it can either emerge as a global leader in AI innovation or risk falling behind in the technology race. The age of AI brings with it immense promise as well as unprecedented challenges, and how India navigates this landscape will shape its socio-economic and geopolitical future.

India's strengths in the AI domain are rooted in its vast pool of skilled engineers, a thriving IT industry, and a growing startup ecosystem. With over 5 million STEM graduates annually and a burgeoning base of AI researchers, India possesses the intellectual capital required to build cutting-edge AI systems. Institutions like IITs, IIITs, and IISc have begun fostering AI research, while private players such as TCS, Infosys, and Wipro are integrating AI into th

RuntimeError: generator raised StopIteration